# Задание 1 (10 баллов)

Обучите языковую модель с помощью GRPO на вот этом датасете - `notbadai/python_functions_reasoning`
Он чем-то похож на датасет из семинара, но вместо математических задач нужно написать код на питоне.
Измените system prompt под эту задачу. Напишите аналогичные reward функции - одна на проверку формата (markdown wrappers ```python... ``` вокруг кода в конце), и на проверку что код работает (можете использовать import ast; ast.parse(string) для проверки)
Если модель сначала не может отвечать в нужно формате, то сделайте цикл SFT обучения на небольшом куске датасета.
GRPO в колабе может работать долго, поэтому не старайтесь пропустить весь датасет. Но обучайте хотя бы на 100 промптах (по несколько генераций на каждый промпт).
Оцените разницу в качестве на небольшом сабсете, прогнав изначальную модель, модель после SFT (если она есть) и GRPO модель. В качестве метрики используйте reward функции.
Можете взять любую языковую модель любого размера. Можете дообучать всю модель или же только адаптера поверх квантизированной модели.


In [1]:
%%capture
!pip install trl
!pip install bitsandbytes

In [2]:
import ast
import numpy as np
import pandas as pd
import re
import torch

from datasets import load_dataset
from peft import LoraConfig, PeftModel
from tqdm.autonotebook import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from trl import GRPOConfig, GRPOTrainer, SFTConfig, SFTTrainer

## Модель

In [3]:
device = "cuda"

In [4]:
model_name = "Qwen/Qwen2-0.5B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


## Датасет

In [5]:
dataset = load_dataset("notbadai/python_functions_reasoning", split="train")
dataset = dataset.shuffle(seed=42)

grpo_dataset = dataset.select(range(0, 150))
sft_dataset = dataset.select(range(150, 2150))
test_dataset = dataset.select(range(2150, 2350))

README.md:   0%|          | 0.00/668 [00:00<?, ?B/s]

train.jsonl:   0%|          | 0.00/414M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/206299 [00:00<?, ? examples/s]

## SFT

### Промпт

In [6]:
system_prompt = """
Your task is to implement functions in Python based on descriptions of their functionality in English.
Before outputting the implementation of the function, give step-by-step reasoning.
Put the final answer at the end preceded by four hash signs like this: "\n####\n*answer*".
The answer should begin with "```python" and end with "```</s>".
"""

def chatml_format(example):
    messages = [{"role": "system", "content": system_prompt},
                {"role": "user", "content": example["prompt"]},
                {"role": "assistant", "content": example["reasoning"] + "\n####\n" + example["answer"]}]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return {
        "text": prompt
    }

### Обучение

In [ ]:
sft_dataset = sft_dataset.map(chatml_format)

In [ ]:
training_args = SFTConfig(
    max_length=2048,
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    logging_steps=10
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    train_dataset=sft_dataset,
    args=training_args
)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: bert-base-multilingual-cased to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,0.859000
20,0.665900
30,0.667300
40,0.642200
50,0.620400
60,0.600200
70,0.616900
80,0.592600
90,0.586100
100,0.560400


TrainOutput(global_step=500, training_loss=0.568023946762085, metrics={'train_runtime': 1261.4646, 'train_samples_per_second': 1.585, 'train_steps_per_second': 0.396, 'total_flos': 2247560058771456.0, 'train_loss': 0.568023946762085})

In [ ]:
trainer.model.push_to_hub("Qwen2-0.5B-Instruct-python-functions-sft")

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/bert-base/Qwen2-0.5B-Instruct-python-functions-sft/commit/a9b6c4d87b48c69c52604adf5c3bee195d5163f6', commit_message='Upload Qwen2ForCausalLM', commit_description='', oid='a9b6c4d87b48c69c52604adf5c3bee195d5163f6', pr_url=None, repo_url=RepoUrl('https://huggingface.co/bert-base/Qwen2-0.5B-Instruct-python-functions-sft', endpoint='https://huggingface.co', repo_type='model', repo_id='bert-base/Qwen2-0.5B-Instruct-python-functions-sft'), pr_revision=None, pr_num=None)

### GRPO

### Модель

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "bert-base/Qwen2-0.5B-Instruct-python-functions-sft",
    torch_dtype=torch.float16,
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16
    )
)

config.json:   0%|          | 0.00/682 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

### Промпт

In [7]:
def chatml_format(example):
    messages = [{"role": "system", "content": system_prompt},
                {"role": "user", "content": example["prompt"]}]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    return {
        "prompt": prompt
    }

In [ ]:
grpo_dataset = grpo_dataset.map(chatml_format)

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

In [ ]:
batch = grpo_dataset[2:5]
input_ids = tokenizer.batch_encode_plus(batch["prompt"], return_tensors="pt", padding=True)

output = model.generate(
    input_ids["input_ids"].to(device), attention_mask=input_ids["attention_mask"].to(device),
    max_new_tokens=1024, do_sample=True, temperature=1.5, pad_token_id=tokenizer.eos_token_id
)

completions = tokenizer.batch_decode(output, skip_special_tokens=False)
print(completions[0])

<|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_start|>system

Your task is to implement functions in Python based on descriptions of their functionality in English.
Before outputting the implementation of the function, give step-by-step reasoning.
Put the final answer at the end preceded by four hash signs like this: "
#

In [8]:
def format_accuracy_func(completions, **kwargs):
    rewards = []

    for response in completions:
        start_index = response.find("```python")
        if start_index == -1:
            rewards.append(0.0)
            continue

        end_index = response.find("```</s>", start_index + 9)
        if end_index == -1:
            rewards.append(0.0)
            continue

        rewards.append(1.0)

    return rewards

In [9]:
def answer_accuracy_func(completions, **kwargs):
    rewards = []

    for response in completions:
        try:
            answer = response.split("####")[-1]
            matched = re.search("```python(.*?)```</s>", answer, re.DOTALL)
            code = matched.group(1).strip()
            ast.parse(code)
            rewards.append(1.0)
        except Exception:
            rewards.append(0.0)

    return rewards

In [ ]:
format_accuracy_func([completions[0]])

[1.0]

In [ ]:
answer_accuracy_func([completions[0]])

[0.0]

In [ ]:
peft_config = LoraConfig(
    r=32,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules="all-linear"
)

In [ ]:
training_args = GRPOConfig(
    output_dir="Qwen/Qwen2-0.5B-Instruct-math-sft-grpo",
    max_completion_length=1024,
    logging_steps=1,
    report_to="none",
    per_device_train_batch_size=3,
    num_generations=3,
    num_train_epochs=1,
    temperature=1.5,
    label_names=["labels"]
)

In [ ]:
trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[format_accuracy_func, answer_accuracy_func],
    args=training_args,
    train_dataset=grpo_dataset,
    peft_config=peft_config

)

In [ ]:
trainer.train()

`generation_config` default values have been modified to match model-specific defaults: {'top_k': 20, 'top_p': 0.8, 'repetition_penalty': 1.1, 'bos_token_id': 151643}. If this is not desired, please set these values explicitly.


Step,Training Loss
1,0.128200
2,-0.145900
3,-0.198200
4,0.186500
5,0.000000
6,-0.027300
7,-0.132500
8,0.185600
9,0.000000
10,0.000000


TrainOutput(global_step=150, training_loss=0.05471393181659702, metrics={'train_runtime': 11788.0976, 'train_samples_per_second': 0.013, 'train_steps_per_second': 0.013, 'total_flos': 0.0, 'train_loss': 0.05471393181659702})

In [ ]:
trainer.model.push_to_hub("Qwen2-0.5B-Instruct-python-functions-sft-grpo")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/70.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/bert-base/Qwen2-0.5B-Instruct-python-functions-sft-grpo/commit/0ce5b4768382e2fadeb6797a0903ab5fc605560f', commit_message='Upload model', commit_description='', oid='0ce5b4768382e2fadeb6797a0903ab5fc605560f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/bert-base/Qwen2-0.5B-Instruct-python-functions-sft-grpo', endpoint='https://huggingface.co', repo_type='model', repo_id='bert-base/Qwen2-0.5B-Instruct-python-functions-sft-grpo'), pr_revision=None, pr_num=None)

## Оценка моделей

In [10]:
def generate_results(model, dataset, save_path, batch_size=2):
    response = []
    dataset.set_format("pandas")

    model.generation_config.temperature = None
    model.generation_config.top_p = None
    model.generation_config.top_k = None

    for i in tqdm(range(0, len(dataset), batch_size)):
        batch = dataset[i:i+batch_size]
        input_ids = tokenizer.batch_encode_plus(
            batch["prompt"].tolist(),
            return_tensors="pt",
            padding=True
        )

        output = model.generate(
            input_ids["input_ids"].to(device),
            attention_mask=input_ids["attention_mask"].to(device),
            max_new_tokens=1024,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )[:,input_ids["input_ids"].shape[-1]:]

        response.extend(tokenizer.batch_decode(output, skip_special_tokens=True))

    df = pd.DataFrame(response, columns=["response"])
    df.to_csv(save_path, index=False)

In [11]:
test_dataset = test_dataset.map(chatml_format)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

### Дефолтная модель

In [12]:
model_default = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2-0.5B-Instruct",
    device_map=device
)

generate_results(model_default, test_dataset, "default_model_responses.csv", batch_size=2)

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [13]:
default_rewards = pd.read_csv("default_model_responses.csv")["response"]

print(
    "Mean format score for the default model:",
    np.mean(format_accuracy_func(default_rewards))
)
print(
    "Mean answer score for the default model:",
    np.mean(answer_accuracy_func(default_rewards))
)

Mean format score for the default model: 0.0
Mean answer score for the default model: 0.0


### Модель после SFT

In [14]:
model_sft = AutoModelForCausalLM.from_pretrained(
    "bert-base/Qwen2-0.5B-Instruct-python-functions-sft",
    device_map=device
)

generate_results(model_sft, test_dataset, "sft_model_responses.csv", batch_size=2)

config.json:   0%|          | 0.00/682 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [15]:
sft_rewards = pd.read_csv("sft_model_responses.csv")["response"]

print(
    "Mean format score for the model after SFT:",
    np.mean(format_accuracy_func(sft_rewards))
)
print(
    "Mean answer score for the model after SFT:",
    np.mean(answer_accuracy_func(sft_rewards))
)

Mean format score for the model after SFT: 1.0
Mean answer score for the model after SFT: 0.47


### Модель после GRPO

In [16]:
model_grpo = PeftModel.from_pretrained(
    model_sft,
    "bert-base/Qwen2-0.5B-Instruct-python-functions-sft-grpo",
    device_map=device
)

generate_results(model_grpo, test_dataset, "grpo_model_responses.csv", batch_size=2)

adapter_config.json:   0%|          | 0.00/824 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/70.4M [00:00<?, ?B/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [17]:
grpo_rewards = pd.read_csv("grpo_model_responses.csv")["response"]

print(
    "Mean format score for the model after GRPO:",
    np.mean(format_accuracy_func(grpo_rewards))
)
print(
    "Mean answer score for the model after GRPO:",
    np.mean(answer_accuracy_func(grpo_rewards))
)

Mean format score for the model after GRPO: 1.0
Mean answer score for the model after GRPO: 0.47
